In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from xlrd import open_workbook
import re
import os
import time
from os import listdir
from os.path import isfile, join
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_datapackage

In [2]:
#out_dir = '../../'
out_dir = '../'

# data to hold all the values
variants = []
#['CONSTANT-FERTILITY', 'ZERO-MIGRATION', 'NOTES', 'HIGH VARIANT', 'ESTIMATES', 'MEDIUM VARIANT', 'NO CHANGE', 'CONSTANT-MORTALITY', 'INSTANT-REPLACEMENT', 'LOW VARIANT']
agegroups = []
age1YrInterval = []
ageBroad = []
age5YrInterval = []
#['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80+', '9', '100+', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '0-4', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '0-1', '0-14', '0-17', '0-19', '0-24', '11-16', '11-17', '11-18', '12-14', '12-16', '12-17', '12-18', '13-17', '13-18', '13-19', '15+', '15-17', '15-24', '15-49', '15-59', '15-64', '16+', '17+', '18+', '18-23', '20+', '20-64', '20-69', '21+', '25+', '25-49', '25-64', '25-69', '3-4', '3-5', '3-6', '4-5', '4-6', '5-10', '5-11', '5-14', '50+', '6-10', '6-11', '6-12', '6-9', '60+', '65+', '7-10', '7-12', '70+', '75+', 'Total', '85+', '90+']
ref_AreaCode = []

In [3]:
#method to read files in a folder...
#onlyfiles = [f for f in listdir('source/byYearInterval') if isfile(join('source/byYearInterval', f))]
#onlyfiles

In [4]:
def createDirectory(Directory):
    if not os.path.exists('../'+Directory.lower()):
        os.makedirs('../'+Directory.lower())

In [5]:
#generate files from the data points.
def GenerateYearFormatFiles(ds_all, Directory, FileNameWithPath, gender, Ref_Area_List):
    
    if (gender.upper() == 'TOTAL') or (gender.upper() == "NA"):
        ds_all = ds_all.drop('gender', axis=1)

    for geo, idxs in ds_all.groupby(by='ref_area_code').groups.items():
        myDS = ds_all.ix[idxs]

        #print(myDS.head(2))
        
        if("agebroad" in Directory.lower()):
            myDS = myDS.rename(columns={
                'age': 'agebroad'
            })
        elif("age1yearinterval" in Directory.lower()):
            myDS = myDS.rename(columns={
                'age': 'age1yearinterval'
            })
        elif("age5yearinterval" in Directory.lower()):
            myDS = myDS.rename(columns={
                'age': 'age5yearinterval'
            })
        
        
        data = Ref_Area_List.loc[Ref_Area_List['Code'] == geo]
        #print(data)
        #print(data.iloc[0]['is--world'])
        if(data.iloc[0]['is--world'] == 1):
            NewDirectory = Directory.replace('ref_area_code','global')
            NewFileNameWithPath = FileNameWithPath.replace('ref_area_code','global')
            myDS = myDS.rename(columns={
                'ref_area_code': 'global'
            })
            #print(NewDirectory)
        elif (data.iloc[0]['is--region'] == 1):
            NewDirectory = Directory.replace('ref_area_code','region')
            NewFileNameWithPath = FileNameWithPath.replace('ref_area_code','region')
            myDS = myDS.rename(columns={
                'ref_area_code': 'region'
            })
            #print(NewDirectory)
        elif (data.iloc[0]['is--continent'] == 1):
            NewDirectory = Directory.replace('ref_area_code','continent')
            NewFileNameWithPath = FileNameWithPath.replace('ref_area_code','continent')
            myDS = myDS.rename(columns={
                'ref_area_code': 'continent'
            })
            #print(NewDirectory)
        elif (data.iloc[0]['is--country'] == 1):
            NewDirectory = Directory.replace('ref_area_code','country')
            NewFileNameWithPath = FileNameWithPath.replace('ref_area_code','country')
            myDS = myDS.rename(columns={
                'ref_area_code': 'country'
            })
        #print(myDS.head(2))

        createDirectory(NewDirectory)
        path = os.path.join(out_dir,NewDirectory+'/'+NewFileNameWithPath.format(geo))
        
        #path = os.path.join(out_dir,NewFileNameWithPath.format(geo))
        
        #to_save = ds_all.ix[idxs]
        #to_save.ix[idxs].to_csv(path, index=False, float_format='%.15g')
        myDS.to_csv(path, index=False, float_format='%.15g')

In [6]:
def load_Files(source, variant, gender, TypeBy):
    #load file using pandas
    data = pd.read_excel(source, sheetname= variant, skiprows=16, na_values='…')
    data = data.drop(['Index', 'Notes'], axis = 1)
    
    #remove the empty space and convert '-' to '_' char
    data['Variant'] = data['Variant'].str.lower().replace(' ', '').replace('-','')
    
    #rename country column and country code column
    data = data.rename(columns={
        'Major area, region, country or area *': 'Ref_Area',
        'Country code': 'Ref_Area_Code'
    })
    #year column is present in Age Type sheets
    if (TypeBy == "Age"):
        data = data.rename(columns={
        'Reference date (as of 1 July)': 'Year'
    })
    #insert Gender column
    data.insert(3, 'Gender', gender)
    if (TypeBy == "YearInterval"):
        data.insert(3, 'Freq', '5Yearly')
    elif (TypeBy == "AgeYearInterval"):
        data = data.rename(columns={
        'Period': 'Year'
        })
        #get first year in yyyy-yyyy time period
        data['Year'] = data['Year'].str[:4] 
        data.insert(3, 'Freq', '5Yearly')

    #update teh AreaCode entity list
    global ref_AreaCode
    ref_AreaCode = ref_AreaCode + list(set(data['Ref_Area_Code']) - set(ref_AreaCode))

    return data
    

In [7]:
def GetDataFromWorkBookSheets(source, gender, indicator, TypeBy):
    all_variants = []

    wbb = open_workbook(filename = source)
    #add variant concepts
    global variants
    variants = variants + list(set(wbb.sheet_names()) - set(variants))
    
    #iterate through each SHEET except "NOTES"
    for sheetName in wbb.sheet_names():
        if(sheetName == 'NOTES'):
            #ignore NOTES sheet since we are not collecting metadata yet
            continue
        else:
            #first load the files
            mydata = load_Files(source, sheetName, gender, TypeBy)
            mydata = mydata.drop(['Ref_Area'], axis=1)
            #based on File format type apply the index and set the column value
            if (TypeBy == "Age"):
                mydata = mydata.set_index(['Ref_Area_Code','Year','Variant','Gender'])
                mydata.columns.name = 'Age'
            elif (TypeBy == "Year"):
                mydata = mydata.set_index(['Ref_Area_Code','Variant', 'Gender'])
                mydata.columns.name = 'Year'
            elif (TypeBy == "YearInterval"):
                mydata = mydata.set_index(['Ref_Area_Code','Variant', 'Gender', 'Freq'])
                mydata.columns.name = 'Year'
            elif (TypeBy == "AgeYearInterval"):
                mydata = mydata.set_index(['Ref_Area_Code','Year','Variant','Gender', 'Freq'])
                mydata.columns.name = 'Age'
            
            mydata = mydata.stack().reset_index().rename(columns={0:indicator})
            all_variants.append(mydata)
            
            #break
    return all_variants

In [8]:
#method to sort the file with refArea, Year, Variant, Gender
def sortDataSets(dsSet_all, TypeBy, FileName):
    dataSet = pd.concat(dsSet_all, ignore_index=True) 
    dataSet.columns = list(map(to_concept_id, dataSet.columns))
    #global agegroups
    global age1YrInterval
    global ageBroad
    global age5YrInterval
    
    if (TypeBy == "Age"): 
        #update the global age groups with any new values
        #print(FileName)
        #print(("age_annual" in FileName.lower()))
        if("broad_age" in FileName.lower()):
            ageBroad = ageBroad + list(set(dataSet['age'].unique()) - set(ageBroad))
        elif("age_annual" in FileName.lower()):
            age1YrInterval = age1YrInterval + list(set(dataSet['age'].unique()) - set(age1YrInterval))
        elif("_age_" in FileName.lower()):
            age5YrInterval = age5YrInterval + list(set(dataSet['age'].unique()) - set(age5YrInterval))
        dataSet.loc[dataSet['age'] == "80+", 'age'] = '80plus'
        dataSet.loc[dataSet['age'] == "100+", 'age'] = '100plus'
        
        dataSet['age'] = [ x.replace('-','_').replace('+','plus').replace('Total','total').strip() for x in dataSet['age']]
        dataSet['age'] = dataSet['age'].astype('category', categories=list(dataSet['age'].unique()), ordered=True)
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','age','variant', 'gender'])
    elif (TypeBy == "Year"):
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','variant', 'gender'])
    elif (TypeBy == "YearInterval"):
        dataSet['year'] = dataSet['year'].str[:4]
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','variant', 'gender', 'freq'])
    elif (TypeBy == "AgeYearInterval"):
        #agegroups = agegroups + list(set(dataSet['age'].unique()) - set(agegroups))
        #update the global age groups with any new values
        if("broad_age" in FileName.lower()):
            ageBroad = ageBroad + list(set(dataSet['age'].unique()) - set(ageBroad))
        elif("age_annual" in FileName.lower()):
            age1YrInterval = age1YrInterval + list(set(dataSet['age'].unique()) - set(age1YrInterval))
        elif("_age_" in FileName.lower()):
            age5YrInterval = age5YrInterval + list(set(dataSet['age'].unique()) - set(age5YrInterval))
        dataSet['age'] = [ x.replace('-','_').replace('+','plus').replace('Total','total').strip() for x in dataSet['age']]
        
#         print(dataSet.head(10))
#         dataSet.loc[dataSet['age'] == "80+", 'age'] = '80plus'
#         dataSet.loc[dataSet['age'] == "100+", 'age'] = '100plus'
        
        dataSet['age'] = dataSet['age'].astype('category', categories=list(dataSet['age'].unique()), ordered=True)
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','age','variant', 'gender', 'freq'])
   
    return dataSet

In [10]:
def updateConceptDF(df, myDSvals, Indicator):
    if(Indicator.lower() == "population"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'population by age group (thousands)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'population by age group, major area, region and country, 1950-2100 (thousands)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "lifeexpectancy"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'life expectancy, e(x), at exact age x (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'life expectancy at exact age, e(x), by major area, region and country, 1950-2100'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "lifeexpectancyat15"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'life expectancy at age 15 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'life expectancy at age 15 by major area, region and country, 1950-2100 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "lifeexpectancyat60"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'life expectancy at age 60 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'life expectancy at age 60 by major area, region and country, 1950-2100 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "lifeexpectancyat80"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'life expectancy at age 80 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'life expectancy at age 80 by major area, region and country, 1950-2100 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "lifeexpectancyatbirth"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'life expectancy at birth (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'life expectancy at birth by major area, region and country, 1950-2100 (years)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "mortality15to50per1000aliveat15"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'adult mortality between age 15 and 50, 35q15 (deaths under age 50 per 1,000 alive at age 15)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'probability of dying between the ages of 15 and 50 years by major area, region and country, 1950-2100 (deaths under age 50 per 1,000 alive at age 15)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "mortality15to60per1000aliveat15"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'adult mortality between age 15 and 60, 45q15 (deaths under age 60 per 1,000 alive at age 15)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'probability of dying between the ages of 15 and 60 years by major area, region and country, 1950-2100 (deaths under age 60 per 1,000 alive at age 15)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "percentagetotaldeaths"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'Percentage of deaths by age group (per 100 (male/female/all) total population)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'Percentage of deaths by age group, major area, region and country, 1950-2100'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "percentagetotalpopulation"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'Percentage of population by age group (per 100 (male/female/all) total population)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'Percentage population by age group, major area, region and country, 1950-2100'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "sexratio_maleper100female"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'Sex ratio by age group (males per 100 females by age group)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'Sex ratio by age group, major area, region and country, 1950-2100 (males per 100 females by age group)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "survivorage"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'life table survivors, l(x), at exact age (x)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'life table survivors at exact age, l(x), by major area, region and country, 1950-2100'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "totaldeaths"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'Number of deaths (thousands)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'deaths, by major area, region and country, 1950-2100 (thousands)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "under40mortalityper1000livebirth"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'under-forty mortality, 40q0 (deaths under age 40 per 1,000 live births)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'probability of dying between birth and the age of 40 years by major area, region and country, 1950-2100 (deaths under age 40 per 1,000 live births)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    elif(Indicator.lower() == "under60mortalityper1000livebirth"):
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = 'under-sixty mortality, 60q0 (deaths under age 60 per 1,000 live births)'
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = 'probability of dying between birth and the age of 60 years by major area, region and country, 1950-2100 (deaths under age 60 per 1,000 live births)'
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    else:
        df.loc[(df['concept'] == Indicator.lower()), 'name'] = myDSvals[1].iloc[6]
        df.loc[(df['concept'] == Indicator.lower()), 'description'] = myDSvals[0].iloc[0]
        df.loc[(df['concept'] == Indicator.lower()), 'sourceurl'] = 'https://esa.un.org/unpd/wpp/DVD/'
    return df

In [11]:
def updateMetaData(df, TypeBY, Directory, FileName, Indicator):
    if(";" in FileName):
        #then get first filename
        files = FileName.split(";")
        FileName = files[0]
    
    #read the excel file to get indicator and description
    if(TypeBY == 'Age'or TypeBY == "AgeYearInterval"):
        myDSvals = pd.read_excel("source/"+FileName, sheetname='ESTIMATES', skiprows=9, nrows=16 ,  header=None, parse_cols = "A,G")
        df = updateConceptDF(df, myDSvals, Indicator)
    else:
        myDSvals = pd.read_excel("source/"+FileName, sheetname='ESTIMATES', skiprows=9, nrows=16 ,  header=None, parse_cols = "A,F")
        df = updateConceptDF(df, myDSvals, Indicator)
    return df
    #print(df)
               

In [12]:
#MAIN Function. Calls the metadata.xslx file and iterate through each file
#supports reading multiple files and concatenating them to one dataset as well.
def callDataPointFiles(metadata_df, cdf):
    Ref_Area_List = pd.read_excel('source/countrymetadata.xlsx', parse_cols = "A:G")
    df = metadata_df
    MainStart = time.time()
    for i, row in enumerate(metadata_df.values):
        start = time.time()
        
        #date = metadata_df.index[i]
        FileName, TypeBY, SEX, Indicator, Directory, name, description, url = row
        newFileName = Directory.lower()
        newFileName = newFileName.replace("ref_area_code", "ref_area_code-{}")
        newFileName = newFileName + ".csv"

        #if(TypeBY == "Year"):
        #if (TypeBY == "Age" and Indicator.lower() == "feminityrato_femaleper100male"):
        if (TypeBY == "Year" or TypeBY == "Age" or TypeBY == "YearInterval" or TypeBY == "AgeYearInterval"):
            print( str(i+1) + ' of '+ str(metadata_df.shape[0]) +' -- ' + FileName)
            
            #update concept name & description in metadata file
            cdf = updateMetaData(cdf, TypeBY, Directory, FileName, Indicator)
            
            #check if FileName has two or more files. for ex same dimensions with male and female
            if(";" in FileName):
                #if yes then run the method for each files and merge the two dataset.
                files = FileName.split(";")
                ds_allSex = []
                #iterate through files then
                for file in files:
                    if "_MALE" in file:
                        SEX = "male"
                    elif "_FEMALE" in file:
                        SEX = "female"
                    ds_sex = GetDataFromWorkBookSheets("source/"+file, SEX, Indicator, TypeBY) 
                    ds_allSex.append(ds_sex)
                    
                #break
                #concat all the sheets and files together as one list
                mainds = []
                for dss in ds_allSex:
                    for dss1 in dss:
                        mainds.append(dss1)
                dataSet = sortDataSets(mainds, TypeBY, FileName)
            else:
                ds_sex = GetDataFromWorkBookSheets("source/"+FileName, SEX, Indicator, TypeBY) 
                dataSet = sortDataSets(ds_sex, TypeBY, FileName)

            #put 3 decimal data for age format. Since some files have data for 80+ as 3.4439999 and 3.444 which
            #is leading to the duplicate error at validate-ddf
#             if (TypeBY == "Age"): 
                #dataSet.loc[dataSet['age'] == "80+", Indicator.lower()] = np.round(dataSet[Indicator.lower()],decimals=3)
                
            dataSet = dataSet.drop_duplicates()

            #create files if df is not empty
            if not dataSet.empty:
                GenerateYearFormatFiles(dataSet, Directory.lower(), newFileName, SEX, Ref_Area_List)
                #GenerateYearFormatFiles(dataSet, newFileName, SEX)
            else:
                print ('File with no data?? Something WRONG?: ' + FileName )
            
            end = time.time()            
            print ('Total Time Taken: ' + str(end-start)) 
            #break
    #break
    print ('Total Time For All Files: ' + str(end-MainStart))
    return cdf

In [13]:
# all functions here are to generate the concepts and entities files.
def generateEntities_Gender():
    cdf = pd.DataFrame([], columns=['gender'])
    cdf['gender'] = ['male', 'female']
    
    path = os.path.join(out_dir, 'ddf--entities--gender.csv')
    cdf.to_csv(path, index=False)

def generateEntities_Freq():
    cdf = pd.DataFrame([], columns=['freq'])
    cdf['freq'] = ['5yearly']
    
    path = os.path.join(out_dir, 'ddf--entities--freq.csv')
    cdf.to_csv(path, index=False)
def generateEntities_Variant():
    variant = [x.lower().replace(' ', '').replace('-', '') for x in variants if x != 'NOTES']
    
    cdf = pd.DataFrame([], columns=['variant'])
    cdf['variant'] = variant
    
    path = os.path.join(out_dir, 'ddf--entities--variant.csv')
    cdf.to_csv(path, index=False)
def generateEntities_AgeGroups():
    cdf = pd.DataFrame([], columns=['agebroad','is--agebroad'])
    cdf['agebroad'] = [ x.replace('-','_').replace('+','plus').replace('Total','total').strip() for x in sorted(set(ageBroad))]
    cdf['is--agebroad'] = 'TRUE'    
    path = os.path.join(out_dir, 'ddf--entities--age--agebroad.csv')
    cdf.to_csv(path, index=False)
    
    cdf1 = pd.DataFrame([], columns=['age1yearinterval','is--age1yearinterval'])
    #print(age1YrInterval)
    cdf1['age1yearinterval'] = [ x.replace('-','_').replace('+','plus').replace('Total','total').strip() for x in sorted(set(age1YrInterval))]
    cdf1['is--age1yearinterval'] = 'TRUE'    
    path = os.path.join(out_dir, 'ddf--entities--age--age1yearinterval.csv')
    cdf1.to_csv(path, index=False)
    
    cdf2 = pd.DataFrame([], columns=['age5yearinterval','is--age5yearinterval'])
    cdf2['age5yearinterval'] = [ x.replace('-','_').replace('+','plus').replace('Total','total').strip() for x in sorted(set(age5YrInterval))]
    cdf2['is--age5yearinterval'] = 'TRUE'    
    path = os.path.join(out_dir, 'ddf--entities--age--age5yearinterval.csv')
    cdf2.to_csv(path, index=False)

    
def generateEntities_RefAreaCode():
    Ref_Area_List = pd.read_excel('source/countrymetadata.xlsx', parse_cols = "A:G")
    
    cdf = pd.DataFrame([], columns=['country', 'name','is--country', 'parent'])
    cdf['country'] = Ref_Area_List.loc[Ref_Area_List['is--country'] == 1, 'Code']
    cdf['name'] = Ref_Area_List.loc[Ref_Area_List['is--country'] == 1, 'Region']
    cdf['parent'] = Ref_Area_List.loc[Ref_Area_List['is--country'] == 1, 'Parent']
    cdf['is--country'] = 'TRUE'    
    path = os.path.join(out_dir, 'ddf--entities--geo--country.csv')
    cdf.to_csv(path, index=False)
    
    cdf1 = pd.DataFrame([], columns=['region','name','is--region','parent'])
    cdf1['region'] = Ref_Area_List.loc[Ref_Area_List['is--region'] == 1, 'Code']
    cdf1['name'] = Ref_Area_List.loc[Ref_Area_List['is--region'] == 1, 'Region']
    cdf1['parent'] = Ref_Area_List.loc[Ref_Area_List['is--region'] == 1, 'Parent']
    cdf1['is--region'] = 'TRUE'
    path = os.path.join(out_dir, 'ddf--entities--geo--region.csv')
    cdf1.to_csv(path, index=False)
    
    cdf2 = pd.DataFrame([], columns=['continent','name','is--continent','parent'])
    cdf2['continent'] = Ref_Area_List.loc[Ref_Area_List['is--continent'] == 1, 'Code']
    cdf2['name'] = Ref_Area_List.loc[Ref_Area_List['is--continent'] == 1, 'Region']
    cdf2['parent'] = Ref_Area_List.loc[Ref_Area_List['is--continent'] == 1, 'Parent']
    cdf2['is--continent'] = 'TRUE'
    path = os.path.join(out_dir, 'ddf--entities--geo--continent.csv')
    cdf2.to_csv(path, index=False)
    
    cdf3 = pd.DataFrame([], columns=['global','name','is--global'])
    cdf3['global'] = Ref_Area_List.loc[Ref_Area_List['is--world'] == 1, 'Code']
    cdf3['name'] = Ref_Area_List.loc[Ref_Area_List['is--world'] == 1, 'Region']
    cdf3['is--global'] = 'TRUE'
    path = os.path.join(out_dir, 'ddf--entities--geo--global.csv')
    cdf3.to_csv(path, index=False)
    
def createConceptsDF(metadata):
    concept_name = ['Year', 'Age', 'Gender', 'Freq','Variant' ,'Geo', 'Country', 
                    'Region', 'Continent', 'Global','AgeBroad', 'Age1YearInterval','Age5YearInterval'
                   ,'name', 'parent', 'domain','description', 'sourceurl']
    
    concept_name = concept_name + list(metadata.Indicator.unique())
    #print(concept_name)
    
    # 53 print(len(list(metadata.Indicator.unique())))
    
    concepts = list(map(to_concept_id, concept_name))
    cdf = pd.DataFrame([], columns=['concept', 'concept_type', 'domain', 'name', 'description', 'sourceurl'])
    cdf['concept'] = concepts
    cdf['concept_type'] = 'measure'
    cdf['concept_type'].iloc[1:6] = 'entity_domain'
    cdf['concept_type'].iloc[6:13] = 'entity_set'
    cdf['concept_type'].iloc[13:18] = 'string'
    cdf['domain'].iloc[6:10] = 'geo'
    cdf['domain'].iloc[10:13] = 'age'
    cdf['concept_type'].iloc[0] = 'time'
    
    return cdf
    
def generateConcepts(cdf):
    path = os.path.join(out_dir, 'ddf--concepts.csv')
    cdf.to_csv(path, index=False)
    

In [14]:
metadata_df = pd.read_excel('source/metadata.xlsx', parse_cols = "A:E")
metadata_df['name'] = ''
metadata_df['description'] = ''
metadata_df['sourceurl'] = 'https://esa.un.org/unpd/wpp/Download/Standard/Population/'

#createConceptDF
cdf = createConceptsDF(metadata_df)
#print (metadata_df)
cdf = callDataPointFiles(metadata_df, cdf)
#print (metadata_df)
#generate concepts
generateConcepts(cdf)

#generate variants
generateEntities_Variant()

#generate countries/world/regions
generateEntities_RefAreaCode()

#generate age groups
generateEntities_AgeGroups()

#generate gender
generateEntities_Gender()

#generate freq
generateEntities_Freq()

#print (concepts)

# add year interval information for freqency/
# split ref_area_code into World/Continent/Region/Country


1 of 79 -- WPP2015_FERT_F03_CRUDE_BIRTH_RATE.XLS
Total Time Taken: 2.1249959468841553
2 of 79 -- WPP2015_POP_F12_A_CHILD_DEPENDENCY_RATIO_1564.XLS
Total Time Taken: 1.8792369365692139
3 of 79 -- WPP2015_POP_F12_B_CHILD_DEPENDENCY_RATIO_2064.XLS
Total Time Taken: 1.9707069396972656
4 of 79 -- WPP2015_POP_F12_C_CHILD_DEPENDENCY_RATIO_2069.XLS
Total Time Taken: 2.1631920337677
5 of 79 -- WPP2015_POP_F12_D_CHILD_DEPENDENCY_RATIO_2564.XLS
Total Time Taken: 2.4656128883361816
6 of 79 -- WPP2015_POP_F12_E_CHILD_DEPENDENCY_RATIO_2569.XLS
Total Time Taken: 2.2592689990997314
7 of 79 -- WPP2015_MORT_F02_CRUDE_DEATH_RATE.XLS
Total Time Taken: 2.4412789344787598
8 of 79 -- WPP2015_POP_F11_A_TOTAL_DEPENDENCY_RATIO_1564.XLS
Total Time Taken: 2.36720609664917
9 of 79 -- WPP2015_POP_F11_B_TOTAL_DEPENDENCY_RATIO_2064.XLS
Total Time Taken: 2.6500961780548096
10 of 79 -- WPP2015_POP_F11_C_TOTAL_DEPENDENCY_RATIO_2069.XLS
Total Time Taken: 2.7106709480285645
11 of 79 -- WPP2015_POP_F11_D_TOTAL_DEPENDENCY_R